In [1]:
import pandas as pd
import os
import re
import requests
import json

# Load and clean BFI box office data

In [2]:
def film_dataframe_prep(folder_path, file_name):
  file_path = os.path.join(folder_path, file_name)

  # Read xls file; ignore first row which contains irrelevant text
  df = pd.read_excel(file_path, skiprows=1)

  # Remove rows for which Rank is null, i.e., rows without box office information
  films = df[~df.Rank.isnull()].reset_index(drop=True)

  # Remove empty columns and columns containing non-box office information
  films.drop(columns=[col for col in films.columns if col.startswith('Unnamed:')],
             inplace=True)

  # Add Source File column
  films['Source File'] = file_name

  # Extract the date using regular expressions
  date_match = re.search(r'(\d{4}-\d{2}-\d{2})', file_name)
  films['Weekend Commencing'] = date_match.group(1)

  return films

In [3]:
# Obtain list of box office figures file names
film_dfs = []
# The path for the folder where the box office figure xls files are stored
folder_path = '/workspaces/box-office-figures/data'

# Iterate through the files in the folder
for file_name in os.listdir(folder_path):
  film_dfs.append(film_dataframe_prep(folder_path, file_name))

In [4]:
# Concatenate data from all files into a dataframe
all_films = pd.concat(film_dfs, ignore_index=True)

The all_films dataframe contains data from all of the files uploaded to the data folder. This would allow an analyst to view how a film has performed at the box office over a period of time.

In [5]:
all_films

,Rank,Film,Country of Origin,Weekend Gross,Distributor,% change on last week,Weeks on release,Number of cinemas,Site average,Total Gross to date,Source File,Weekend Commencing
0,1.0,Deadpool & Wolverine,UK/USA,8021085,Disney,-0.53,2.0,715.0,11218.0,33433823.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02
1,2.0,Despicable Me 4,USA,2561637,Universal,-0.18,4.0,731.0,3504.0,30845825.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02
2,3.0,Twisters,USA,1184698,Warner Bros,-0.21,3.0,692.0,1712.0,10197163.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02
3,4.0,Inside Out 2,USA,939144,Disney,-0.17,8.0,670.0,1402.0,52218943.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02
4,5.0,Harold And The Purple Crayon,USA,641549,Sony Pictures,-,1.0,600.0,1069.0,641549.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...
122,48.0,Boat,Ind,3391,DG Tech,-,1.0,10.0,339.0,3391.0,bfi-weekend-box-office-report-2024-08-09-11.xls,2024-08-09
123,52.0,Coraline (15th Anniversary),USA,3227,Trafalgar,-,1.0,1.0,3227.0,3227.0,bfi-weekend-box-office-report-2024-08-09-11.xls,2024-08-09
124,54.0,Adios Amigo (Ireland),Ind,3078,2G Entertainments,-,1.0,12.0,256.0,3078.0,bfi-weekend-box-office-report-2024-08-09-11.xls,2024-08-09
125,81.0,Devadoothan (Ireland),Ind,953,2G Entertainments,-,1.0,1.0,953.0,953.0,bfi-weekend-box-office-report-2024-08-09-11.xls,2024-08-09


# Extract genre information using the TMDB API

This section of the notebook will not work unless you have an access token generated for the TMDB API. Please see README.md for more information.

## Extract genre list

TMDB gives genre IDs, so we will need to extract the id: genre mapping

In [6]:
# Read access token for the API
api_rat = os.environ['TMDB_READ_ACCESS_TOKEN']

In [7]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + api_rat
}

genres = requests.get(url, headers=headers)

print(genres.text)

{"genres":[{"id":28,"name":"Action"},{"id":12,"name":"Adventure"},{"id":16,"name":"Animation"},{"id":35,"name":"Comedy"},{"id":80,"name":"Crime"},{"id":99,"name":"Documentary"},{"id":18,"name":"Drama"},{"id":10751,"name":"Family"},{"id":14,"name":"Fantasy"},{"id":36,"name":"History"},{"id":27,"name":"Horror"},{"id":10402,"name":"Music"},{"id":9648,"name":"Mystery"},{"id":10749,"name":"Romance"},{"id":878,"name":"Science Fiction"},{"id":10770,"name":"TV Movie"},{"id":53,"name":"Thriller"},{"id":10752,"name":"War"},{"id":37,"name":"Western"}]}


In [8]:
genres_json = json.loads(genres.text)
genres_list = genres_json['genres']
genres_df = pd.DataFrame(genres_list).set_index('id')
genres_df

,name
id,
28,Action
12,Adventure
16,Animation
35,Comedy
80,Crime
99,Documentary
18,Drama
10751,Family
14,Fantasy


## Add genre column to all_films dataframe

In [9]:
def tmdb_film_search(film):
  # This function searches for the film in TMDB
  url = "https://api.themoviedb.org/3/search/movie?query=" + film + "&include_adult=false&language=en-US&page=1"

  headers = {
      "accept": "application/json",
      "Authorization": "Bearer " + api_rat
  }

  response = requests.get(url, headers=headers)
  return response

In [13]:
def find_genre(api_response, genres_df):
  # This function takes a TMDB API response and a dataframe of genres and returns the genre(s) as a string. 
  response_json = json.loads(api_response.text)

  # Returns an empty string if the TMDB search returns zero results
  if response_json['total_results'] == 0:
    genres_string = ""
  else:
    genre_id_list = response_json['results'][0]['genre_ids']
    # Extracts the genre names from genres_df we created earlier
    genres_str_list = genres_df.loc[genre_id_list, 'name'].tolist()
    genres_string = ", ".join(genres_str_list)
  return genres_string

In [14]:
all_films['Genre'] = all_films.Film.apply(tmdb_film_search).apply(find_genre, genres_df=genres_df)

In [15]:
all_films

,Rank,Film,Country of Origin,Weekend Gross,Distributor,% change on last week,Weeks on release,Number of cinemas,Site average,Total Gross to date,Source File,Weekend Commencing,Genre
0,1.0,Deadpool & Wolverine,UK/USA,8021085,Disney,-0.53,2.0,715.0,11218.0,33433823.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02,"Action, Comedy, Science Fiction"
1,2.0,Despicable Me 4,USA,2561637,Universal,-0.18,4.0,731.0,3504.0,30845825.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02,"Animation, Family, Comedy, Action"
2,3.0,Twisters,USA,1184698,Warner Bros,-0.21,3.0,692.0,1712.0,10197163.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02,"Action, Adventure, Drama"
3,4.0,Inside Out 2,USA,939144,Disney,-0.17,8.0,670.0,1402.0,52218943.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02,"Animation, Family, Adventure, Comedy"
4,5.0,Harold And The Purple Crayon,USA,641549,Sony Pictures,-,1.0,600.0,1069.0,641549.0,bfi-weekend-box-office-report-2024-08-02-04.xls,2024-08-02,"Adventure, Family, Fantasy, Comedy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,48.0,Boat,Ind,3391,DG Tech,-,1.0,10.0,339.0,3391.0,bfi-weekend-box-office-report-2024-08-09-11.xls,2024-08-09,"Horror, Drama"
123,52.0,Coraline (15th Anniversary),USA,3227,Trafalgar,-,1.0,1.0,3227.0,3227.0,bfi-weekend-box-office-report-2024-08-09-11.xls,2024-08-09,
124,54.0,Adios Amigo (Ireland),Ind,3078,2G Entertainments,-,1.0,12.0,256.0,3078.0,bfi-weekend-box-office-report-2024-08-09-11.xls,2024-08-09,
125,81.0,Devadoothan (Ireland),Ind,953,2G Entertainments,-,1.0,1.0,953.0,953.0,bfi-weekend-box-office-report-2024-08-09-11.xls,2024-08-09,
